In [1]:
import numpy as np

In [17]:
class MLP:
    def __init__(self,InputSize : int, hiddenLayers: list, OutputSize:int):
        self.weights = {}

        self.weights["W1"] = np.zeros((InputSize,hiddenLayers[0]))
        self.weights["B1"] = np.zeros(hiddenLayers[0])

        wn = 2
        for i in range(1,len(hiddenLayers)):
            self.weights["W"+str(wn)] = np.zeros((hiddenLayers[i-1],hiddenLayers[i]))
            self.weights["B"+str(wn)] = np.zeros(hiddenLayers[i])

            wn += 1

        self.weights["W"+str(wn)] = np.zeros((hiddenLayers[-1],OutputSize))
        self.weights["B"+str(wn)] = np.zeros(OutputSize)

    def activation(self,Z):
        return 1/(1+np.exp(-Z))

    def forward(self,X):

        self.outputs = []
        A = X
        for i in range(1,len(self.weights)//2 + 1):
            Z = A@self.weights["W"+str(i)] + self.weights["B"+str(i)]
            A = self.activation(Z)
            self.outputs.append(A)

        return A
        

In [18]:
clf = MLP(3,[4,3,2],1)

In [19]:
X = np.array([[1,2,3],[4,5,6]])
clf.forward(X)

array([[0.5],
       [0.5]])

In [20]:
clf.outputs

[array([[0.5, 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, 0.5]]),
 array([[0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5]]),
 array([[0.5, 0.5],
        [0.5, 0.5]]),
 array([[0.5],
        [0.5]])]